In [ ]:
!pip install pandas pyarrow

In [ ]:
!git clone https://github.com/nianlonggu/MemSum.git

In [ ]:
import pandas as pd
import json

In [ ]:
df_full_text=pd.read_parquet("../../Dares_1Kaccords_dataset_txt_label.parquet")
df_full_text=df_full_text.drop(columns="label_txt")

In [ ]:
df_summary_label=pd.read_parquet("../../Dares_accords_docx_sommaire_label_230811.parquet")

In [ ]:
df_join_ftext_sum=df_full_text.join(df_summary_label)

In [ ]:
def trim_new_line_on_start(string):
    if string.startswith("\n") or string.startswith(" ")or string.startswith("\t"):
        return trim_new_line_on_start(string[1:])
    else:
        return string

In [ ]:
df_join_ftext_sum["txt_trim"]=df_join_ftext_sum["txt"].apply(trim_new_line_on_start)

In [ ]:
df_join_ftext_sum["txt_trim_list"]=df_join_ftext_sum[["txt_trim"]].apply(lambda x : x.str.split("\n"))

In [ ]:
df_join_ftext_sum["sommaire_label_list"]=df_join_ftext_sum[["sommaire_label"]].apply(lambda x : x.str.split("\n"))

In [ ]:
def return_dict(text,label):
    d=dict()
    d["text"]=text
    d["summary"]=label
    return json.dumps(d)

In [ ]:
df_join_ftext_sum["dict_txt_sommaire"]=df_join_ftext_sum[["txt_trim_list","sommaire_label_list"]].apply(lambda x :return_dict(*x),axis=1)


In [ ]:
df_join_ftext_sum=df_join_ftext_sum.reset_index()[['dict_txt_sommaire']]

In [ ]:
df_join_ftext_sum=df_join_ftext_sum.set_index("dict_txt_sommaire")

In [ ]:
df_join_ftext_sum.to_csv("train_data.json")

In [ ]:
!cat train_data.json | tail -n +2 | sed -e 's/"{/{/g' | sed -e 's/}"/}/g'  | sed -e 's/""/"/g' > train_data_parsed.json

In [ ]:
!pip install -q -r MemSum/requirements.txt

In [ ]:
!cd MemSum/data_preprocessing; python get_high_rouge_episodes_mp.py -input_corpus_file_name ../../train_data_parsed.json -output_corpus_file_name custom_train.json -beamsearch_size 2 -n_processes 71